# API Methode

In [38]:
import tweepy

from datetime import datetime

consumer_key= 'txGKpZibczbB3viciPGaIht1h'
consumer_secret= 'LdBcAwDspNV2IoGdBrDC5wTiprqW4LiHPFTOWcq95a9FgkWc5M'
access_token= '1622613622860967937-HtuI8jqgHy8P5fJgN1EKGLqqDqWCOw'
access_token_secret= 'uEpId3rtrIdNZe9iylsohsu7xvoMHs8NHgfNOndfgh6Me'
bearer_token="AAAAAAAAAAAAAAAAAAAAAJ%2BwlgEAAAAAApXZ%2BQkCOMMCUz8yA0Q2U7cIx4A%3D54C8m6MSpNmM1g4oeEH5ZpzknYFfJ4zYSWSaxVteTpfofyg5Ni"
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

user = client.get_user(username="CBirckner",user_fields=["created_at", "description","id", "location", "name", "pinned_tweet_id", "profile_image_url", "protected", "public_metrics", "url", "username", "verified", "verified_type", "withheld"])

profile_data =  {
                    'name' : user.data.name,
                    'id' : user.data.id,
                    'username' : user.data.username,
                    'created_at' : user.data.created_at,
                    'created_at_year':datetime.strptime(str(user.data.created_at)[:19], '%Y-%m-%d %H:%M:%S').year,
                    'description' : user.data.description,
                    'location' : user.data.location,
                    'profile_image_url' : user.data.profile_image_url,
                    'protected' : user.data.protected,
                    'followers_count' : user.data.public_metrics["followers_count"],
                    'following_count' : user.data.public_metrics["following_count"],
                    'tweet_count' : user.data.public_metrics["tweet_count"],
                     'listed_count' : user.data.public_metrics["listed_count"],
                    'verified' : user.data.verified,
                    'verified_type' : user.data.verified_type,
                       'withheld' : user.data.withheld,
                       'pinned_tweet_id':user.data.pinned_tweet_id
                }
print(profile_data)

tweets = client.get_liked_tweets(id=profile_data['id'], tweet_fields=['context_annotations','created_at','geo'])



{'name': 'Carli Birckner', 'id': 105916557, 'username': 'CBirckner', 'created_at': datetime.datetime(2010, 1, 17, 22, 54, 19, tzinfo=datetime.timezone.utc), 'created_at_year': 2010, 'description': 'Television producer. Emmy Award winner. Disney enthusiast. ZBB followers. Kitty crazy.', 'location': 'Baltimore, MD', 'profile_image_url': 'https://pbs.twimg.com/profile_images/632916759557840896/35puTLF2_normal.jpg', 'protected': False, 'followers_count': 55, 'following_count': 131, 'tweet_count': 270, 'listed_count': 2, 'verified': False, 'verified_type': 'none', 'withheld': None, 'pinned_tweet_id': None}


# scraping methode with selenium

In [6]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options as ChromeOptions
from fake_headers import Headers
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import json
def set_properties(browser_option):
            ua = Headers().generate()      #fake user agent
            browser_option.add_argument('--headless')
            browser_option.add_argument('--disable-extensions')
            browser_option.add_argument('--incognito')
            browser_option.add_argument('--disable-gpu')
            browser_option.add_argument('--log-level=3')
            browser_option.add_argument(f'user-agent={ua}')
            browser_option.add_argument('--disable-notifications')
            browser_option.add_argument('--disable-popup-blocking')

            return browser_option
browser_option = ChromeOptions()
browser_option = set_properties(browser_option)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=browser_option) #chromedriver's path in first argument
print("Loading Page...")
driver.get("https://twitter.com/CBirckner")
print("wait script...")
WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH , "//script[@data-testid='UserProfileSchema-test']")))
print("get html")
html1 = driver.execute_script("return document.documentElement.outerHTML")
# This will get the initial html - before javascript
# html1 = driver.page_source

# parse html content
soup = BeautifulSoup(driver.page_source , 'html.parser')

res=soup.find("head").find('script', attrs={'data-testid': 'UserProfileSchema-test'})

json_object = json.loads(res.contents[0])
print(json_object)

Loading Page...
wait script...
get html
{'@context': 'http://schema.org', '@type': 'ProfilePage', 'dateCreated': '2010-01-17T22:54:19.000Z', 'author': {'@type': 'Person', 'additionalName': 'CBirckner', 'description': 'Television producer. Emmy Award winner. Disney enthusiast. ZBB followers. Kitty crazy.', 'givenName': 'Carli Birckner', 'homeLocation': {'@type': 'Place', 'name': 'Baltimore, MD'}, 'identifier': '105916557', 'image': {'@type': 'ImageObject', 'contentUrl': 'https://pbs.twimg.com/profile_images/632916759557840896/35puTLF2_400x400.jpg', 'thumbnailUrl': 'https://pbs.twimg.com/profile_images/632916759557840896/35puTLF2_normal.jpg'}, 'interactionStatistic': [{'@type': 'InteractionCounter', 'interactionType': 'https://schema.org/FollowAction', 'name': 'Follows', 'userInteractionCount': 55}, {'@type': 'InteractionCounter', 'interactionType': 'https://schema.org/SubscribeAction', 'name': 'Friends', 'userInteractionCount': 131}, {'@type': 'InteractionCounter', 'interactionType': 'h

# scraper methode with snscrape

In [66]:
import snscrape.modules.twitter as sntwitter
data = sntwitter.TwitterUserScraper('CBirckner')._get_entity()
print(type(data))

profile_data =  {
                    'name' : data.displayname,
                    'id' : data.id,
                    'username' : data.username,
                    'created_at' : data.created,
                    'created_at_year':datetime.strptime(str(data.created)[:19], '%Y-%m-%d %H:%M:%S').year,
                    'description' : data.renderedDescription,
                    'location' : data.location,
    
                    'profile_image_url' : data.profileImageUrl,
                    'protected' : data.protected,
                    'followers_count' : data.followersCount,
                    'following_count' : data.friendsCount,
                    'tweet_count' :data.statusesCount,
                    'listed_count' : data.listedCount,
                     'favourites_count' :data.favouritesCount,
                      'media_count' :data.mediaCount,
                    'verified' :data.verified
                }
print(profile_data)




<class 'snscrape.modules.twitter.User'>
{'name': 'Carli Birckner', 'id': 105916557, 'username': 'CBirckner', 'created_at': datetime.datetime(2010, 1, 17, 22, 54, 19, tzinfo=datetime.timezone.utc), 'created_at_year': 2010, 'description': 'Television producer. Emmy Award winner. Disney enthusiast. ZBB followers. Kitty crazy.', 'location': 'Baltimore, MD', 'profile_image_url': 'https://pbs.twimg.com/profile_images/632916759557840896/35puTLF2_normal.jpg', 'protected': False, 'followers_count': 55, 'following_count': 131, 'tweet_count': 270, 'listed_count': 2, 'favourites_count': 1484, 'media_count': 2, 'verified': False}
